# Planejamento do projeto

## Resultado final
    - Uma classe em linguagem python em que o usuário digita o símbolo da ação (stock ticker) e recebe uma série 
    de gráficos e análises da empresa selecionada.
    - Quais análises serão entregues?
        . Gráfico interativo com o comportamento das ações no tempo;
        . Relatório com os principais indicadores;
        . Avaliação de investimento, favorável ou desvaforável?
        . Previsão de comportamento?

## Ferramental
    - Utilizar o Jupyter Notebook para a escrita e interpretação do código;
    - Utilizar a biblioteca Selenium ou urllib.request para analisar conteudos de páginas;
    - Utilizar a biblioteca Beautiful Soup (ou lXml e Scrapy) que torna possível ler o conteúdo das páginas HTML;
    - Utilizar a biblioteca Pandas para armazenar o conteúdo das páginas em DataFrames;
    - Utilizar a biblioteca Numpy para realização de operações;
    - Utilizar a biblioteca Plotly para desenhar gráficos interativos;
    - Utilizar a biblioetca pyplot da matplotlib para criação de gráficos e dashboards;
    - Utilizar a biblioteca sklearn para a adaptação de um modelo de previsão de preço de ação.
    
## Processo de desenvolvimento
    - Coletar os dados das páginas que contêm informações sobre ações: https://finance.yahoo.com/ 
        - Dados históricos dos preços das ações
        - Indicadores fundamentalistas das ações 
        - Dados históricos dos fundos 
        - Indicadores dos fundos
    - Organizar os dados em DataFrames
    - Tratar os dados, isto é, retirar dados duplicados, tratar dados faltantes, ajustar indices e headers
    - Gerar as análises dos dados

# Utilizando o URLLIB

In [ ]:
# Imports para o desenvolvimento do projeto
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request as ur
import requests

In [ ]:
# Endereço de uma das páginas a qual fornecerá informações para a análise
url = "https://br.financas.yahoo.com/quote/{}/history?period1={}&period2={}&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"

# Tratar condições de erro

## Como tratar possíveis condições de erro? 
    - Colocando try except 
    - Colocando assert
    

In [ ]:
# Inserir o nome empresa
ticker = input()

# Selecionado a data do horizonte de busca de 5 anos 
date2 = np.timedelta64(np.datetime64('today') - np.datetime64('1969-12-31'), 's').astype(int)
date1 = date2 - 86400 * 365 * 5

# Formatando o url 
url = url.format(ticker, date1, date2)
print(url)

In [ ]:
# Adicionar cabeçalho para passar informações adicionais. Deve-se identificar o erro retornado pelo navegador (401 por exemplo
# é um exemplo de permissão negada) e adicionar o cabeçalho correspondente, no caso o site necessitava de alguma identificação 
# para concretizar a requisição, essa informação pode ser o tipo de aplicação o sistema operacional, fornecedores, a versão 
# de agente de usuário requisitante e outros. "Mozilla/5.0" é o token geral que diz que o navegador é compatível com Mozilla.
# Por razões históricas, quase todo navegador envia isso hoje. 
# Fonte: https://developer.mozilla.org/pt-BR/docs/Web/HTTP/Headers/User-Agent
     
response = requests.get(url, headers=headers) #baixa o conteúdo de uma página na web
status = response.raise_for_status()

# Retorna um objeto BeautifulSoup que permite manipular o data structure com mais facilidade
soup = BeautifulSoup(response.text, 'html.parser')

# O título da página HTML
print(soup.title.string)

linhas = [] #lista para armazenar os dados retirados do HTML
for element in soup.find_all(['th', 'td', 'strong']):
    linhas.append(element.string) #retira os strings que acompanham as classes selecionadas
    
# Tratando os strings para obter o dado desejado
linhas = [e for e in linhas if e not in ('Operating Expenses','Non-recurring Events')]
new_lista = list(filter(None,linhas)) #retira os Nones

dado = list(zip(*[iter(new_lista)]*7))
df_dado = pd.DataFrame(dado)

# Raspagem da tabela de preços de ações utilizando o Selenium

In [2]:
# Imports
import time
import requests
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import numpy as np

In [140]:
# Ajuste do url
# inserir o nome empresa
ticker = "XPLG11.SA"

# selecionado a data do horizonte de busca de 5 anos 
date2 = np.timedelta64(np.datetime64('today') - np.datetime64('1969-12-31'), 's').astype(int)
date1 = date2 - 86400 * 365 * 5

url = "https://br.financas.yahoo.com/quote/{}/history?period1={}&period2={}&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"

# formatando o url da página que será acessada
url = url.format(ticker, date1, date2)

# Instância do navegador
opt = webdriver.ChromeOptions()
opt.headless = True #não mostrar a ação em andamento 
ser = Service(r'C:\Program Files (x86)\Google\Chrome\Application\97.0.4692.99\chromedriver.exe')
driver = webdriver.Chrome(service=ser, options=opt)

driver.get(url)

time.sleep(5)

html = driver.find_element(By.TAG_NAME, 'html')

# posição do inicial do scroll
last_height = driver.execute_script("return document.body.scrollHeight")
print("last_height: {}".format(last_height))

while (True):
    # posiciona o scroll no final da página
    html.send_keys(Keys.END)
    
    # pausa para carregar a página
    time.sleep(1)

    # atualiza a posição do scroll 
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    print("new_height: {}".format(new_height))
    
    # verifica se houve movimento da página 
    if (new_height == last_height):
        # termina o loop
        break
    else:
        # atualiza o último valor 
        last_height = new_height         

element = driver.find_element(By.TAG_NAME, 'table')
html_content = element.get_attribute('outerHTML')

driver.quit()

# Código HTML da tabela
soup = BeautifulSoup(html_content, 'html.parser')

# Tratando o HTML para gerar a tabela de hostórico do preço da ação
# lista para armazenar o dados que não sejam relacionados com o preço da ação (dividendo e desdobramento)
list_not_related = [] #lista com conteúdo não relacionado com o preço das ações
tuplas_eventos = [] #tuplas data, valor, ocorrencia (dividendo e desdobramento)
for element in soup.find_all('td',"Ta(start) Py(10px)"):
    list_not_related.append(element)
    tuplas_eventos.append((element.previous_element, element.find('strong').string, element.find('span').string))

linhas_tab = [] #lista para armazenar os dados das linhas da tebela
# retira os strings que acompanham as classes selecionadas e filtra os casos não queridos
for element in soup.find_all(['td', 'th']):
    if element in list_not_related:
        del(linhas_tab[-1]) #deleta o elemento anterior
    else:
        linhas_tab.append(element.string)
        
new_lista = list(filter(None, linhas_tab)) #retira os Nones se existirem
stock_data = list(zip(*[iter(new_lista)]*7)) #empacota em listas de 7 elementos 

df_data_stocks = pd.DataFrame(stock_data[1:], columns=stock_data[0][0:7]) 
df_not_related = pd.DataFrame(tuplas_eventos, columns=['Data', 'Valor', 'Tipo'])


# Tratando o DataFrame
# converte a última coluna para inteiro substituindo o ponto
df_data_stocks["Volume"] = df_data_stocks['Volume'].apply(lambda x: int(x.replace(".","")) if x != "-" else x)

# laço para substituir os pontos por vírgulas 
for coluna in df_data_stocks.columns[1:6]:
    df_data_stocks[coluna] = df_data_stocks[coluna].apply(lambda x: float(x.replace(",",".")) if x != '-' else x)

# dicionário para auxiliar na correção das datas
dicio = {"jan.": "01",
         "fev.": "02",
         "mar.": "03",
         "abr.": "04",
         "mai.": "05",
         "jun.": "06",
         "jul.": "07",
         "ago.": "08",
         "set.": "09",
         "out.": "10",
         "nov.": "11",
         "dez.": "12"}

# ordenando cada data por Ano-mês-dia, substituindo o nome do mês pelo número correspondente e convertendo para datetime64
df_data_stocks["Data_Time"] = pd.to_datetime(df_data_stocks["Data"].apply(lambda x: x.split(' ')[0:6:2][::-1]).\
                                             apply(lambda x: "".join([x[0], dicio[x[1]], x[2]])), 
                                             format='%Y-%m-%d')

# Rearanjando as colunas do dataframe
cols = df_data_stocks.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_data_stocks = df_data_stocks[cols]
df_data_stocks.index = df_data_stocks['Data_Time'] #faz o índice virar a coluna data_time
df_data_stocks.drop(labels='Data_Time', axis=1)

df_data_stocks

last_height: 583
new_height: 8316
new_height: 12105
new_height: 15894
new_height: 16018
new_height: 16018


,Data_Time,Data,Abrir,Alto,Baixo,Fechamento*,Fechamento ajustado**,Volume
Data_Time,,,,,,,,
2022-01-20,2022-01-20,20 de jan. de 2022,0.00,0.00,0.00,100.10,100.10,-
2022-01-19,2022-01-19,19 de jan. de 2022,101.00,101.99,100.18,101.80,101.80,60057
2022-01-18,2022-01-18,18 de jan. de 2022,99.37,100.85,99.04,100.85,100.85,41217
2022-01-17,2022-01-17,17 de jan. de 2022,97.79,99.90,97.33,99.37,99.37,41972
2022-01-14,2022-01-14,14 de jan. de 2022,97.20,97.90,96.84,97.80,97.80,63940
...,...,...,...,...,...,...,...,...
2020-06-10,2020-06-10,10 de jun. de 2020,119.06,121.95,118.97,120.81,120.81,65441
2020-06-09,2020-06-09,09 de jun. de 2020,118.49,118.97,117.00,118.97,118.97,93157
2020-06-08,2020-06-08,08 de jun. de 2020,117.36,119.00,117.21,118.50,118.50,125987


# Tabela de indicadores de FII

In [35]:
url = 'https://www.fundsexplorer.com.br/ranking'
header ={'user-agent':'Mozilla/5.0'}
lendo_url = requests.get(url, headers = header)

lendo_url2 = lendo_url.text
# lendo_url2
soup = BeautifulSoup(lendo_url.content,'html.parser')
listona = []

for i in soup.find_all('td'):
    listona.append(i.string)
indice = 0
listona_separado_linhas = []
while indice < len(listona):
    lista_provisoria=[]
    for z in range(indice,indice+25):
        lista_provisoria.append(listona[z])
    listona_separado_linhas.append(lista_provisoria)
    indice += 26
# listona_separado_linhas
df_fundos = pd.DataFrame(listona_separado_linhas, columns =['Código do fundo', 'Setor', 'Preço atual','Liquidez','Dividendo','Dividend Yield','DY 3M','DY 6M','DY 12M','DY 3M media','DY 6M media','DY 12M media','DY ANO','Variação Preço','Rentab. Período','Retab. Acum.','Patrimônio Líq.','VPA','P/VPA','DY PATR.','Varia. Patri.','Rentab. Patr.', 'Vacância Física','Vacância Financeira','Quantidade de ativos'])
df_fundos

df_fundos.to_excel("Lista_indicadores_fundo.xlsx")

# Tabela de indicadores fundamentalistas

In [36]:
import pandas as pd
import ssl

ssl._create_default_https_context = ssl._create_unverified_context 

url_link = "http://br.financas.yahoo.com/quote/SULA4.SA/key-statistics?p=SULA4.SA" 

r = requests.get(url_link, headers ={'User-Agent':'Mozilla/5.0'})
read_html_pandas_data = pd.read_html(r.text)
